In [1]:
from ConectionSQL import SQL
import os
import pandas as pd

In [2]:
conecta=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=conecta.ConexaoSQL()

In [3]:
querys={

'Falta':

"""

WITH TabTpCorte (cd_tp_faltaprd,descrica) AS (

SELECT cd_tp_faltaprd,descricao
FROM tp_faltaprd
WHERE ativo=1
--AND descricao LIKE '%CORTE%'
),

TabFalta (dt_falta,cd_vend,cd_clien,nu_ped,cd_prod,unid_vda,qtde_falta_vda,preco_unit,vl_total) AS (

SELECT CONVERT(DATETIME,CAST(dt_falta AS date),101) AS dt_falta,
cd_vend,cd_clien,nu_ped,cd_prod,unid_vda,qtde_falta,preco_unit,
ROUND(qtde_falta*preco_basico,2) AS vl_total
FROM faltaprd
INNER JOIN TabTpCorte ON faltaprd.cd_tp_faltaprd=TabTpCorte.cd_tp_faltaprd
)

SELECT dt_falta AS 'Data de Falta',
TabFalta.cd_vend AS 'ID Vendedor',
TabFalta.cd_clien AS 'ID Cliente',cliente.nome_res AS 'Nome Fantasia',
nu_ped AS 'Pedido',TabFalta.cd_prod AS 'SKU',produto.descricao AS 'Produto',
unid_vda AS 'Unid. VDA',qtde_falta_vda AS 'Qtde. VDA',
preco_unit AS 'Valor Unitário',vl_total AS 'Total do Pedido'
FROM TabFalta
--INNER JOIN vendedor ON TabFalta.cd_vend=vendedor.cd_vend
INNER JOIN cliente ON TabFalta.cd_clien=cliente.cd_clien
INNER JOIN produto ON TabFalta.cd_prod=produto.cd_prod
WHERE YEAR(dt_falta)=YEAR(GETDATE()) AND MONTH(dt_falta)=MONTH(GETDATE())
AND DAY(dt_falta)=DAY(GETDate())

""",

'Supervisor':

"""

WITH TabSupervisor (cd_emp,cd_equipe,descricao,cd_vend_sup,nome_resumido,email) AS (

SELECT equipe.cd_emp,equipe.cd_equipe,equipe.descricao,equipe.cd_vend_sup,
CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))=0 THEN vendedor.nome ELSE
RTRIM(LEFT(vendedor.nome,(CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))))) + ' ' + 
LTRIM(RIGHT(vendedor.nome,(CHARINDEX(' ',REVERSE(LTRIM(RTRIM(vendedor.nome))))))) END AS nome_resumido,
usuario.e_mail
FROM equipe
INNER JOIN vendedor ON equipe.cd_vend_sup=vendedor.cd_vend AND equipe.cd_emp=vendedor.cd_emp
LEFT JOIN usuario ON vendedor.cd_vend=usuario.cd_usuario
),

TabGerencia (cd_emp,cd_vend_sup,nome_resumido,email) AS (

SELECT DISTINCT equipe.cd_emp,equipe.cd_vend_sup,TabSupervisor.nome_resumido,TabSupervisor.email
FROM equipe
INNER JOIN TabSupervisor ON equipe.cd_vend_sup=TabSupervisor.cd_vend_sup 
AND equipe.cd_emp=TabSupervisor.cd_emp
WHERE equipe.cd_equipe LIKE '%N%' AND ativo=1
),

TabEquipes (cd_equipe) AS (

SELECT DISTINCT cd_equipe
FROM ped_vda
INNER JOIN tp_ped ON ped_vda.tp_ped=tp_ped.tp_ped AND tp_ped.estat_com=1
INNER JOIN vendedor ON ped_vda.cd_vend=vendedor.cd_vend AND vendedor.ativo=1
)

SELECT * FROM (
SELECT TabSupervisor.cd_equipe AS 'ID Equipe',TabSupervisor.descricao AS 'Equipe',
TabSupervisor.nome_resumido AS 'Supervisor',
TabSupervisor.email AS 'E-mail Sup',
TabGerencia.nome_resumido AS 'Gerente',TabGerencia.email AS 'E-mail Gerg',
CASE WHEN TabSupervisor.nome_resumido=TabGerencia.nome_resumido 
THEN 'N' ELSE 'S' END AS 'Diferente'
FROM TabSupervisor
INNER JOIN TabGerencia ON TabSupervisor.cd_emp=TabGerencia.cd_emp
INNER JOIN TabEquipes ON TabSupervisor.cd_equipe=TabEquipes.cd_equipe
)equipes
ORDER BY 1

""",
    
'Vendedor':
    
"""

SELECT V.CD_VEND AS 'ID Vendedor',V.NOME AS 'Vendedor',
CASE WHEN 
CHARINDEX(' ',LTRIM(RTRIM(V.NOME)))=0 THEN V.NOME ELSE
LTRIM(RTRIM(LEFT(V.NOME,CHARINDEX(' ',LTRIM(RTRIM(V.NOME))))))+ ' '+
LTRIM(RTRIM(RIGHT(V.NOME,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(V.NOME))))))) 
END AS 'Nome Resumido',V.CD_EQUIPE AS 'ID Equipe',
U.E_MAIL AS 'E-mail',categ.descricao AS 'Categoria'
FROM VENDEDOR V
LEFT JOIN USUARIO U ON V.CD_VEND=U.CD_USUARIO
LEFT JOIN categ ON V.CATEG=categ.categ
WHERE V.ATIVO=1


"""

}

In [4]:
falta_df=pd.read_sql(querys['Falta'],conectando)

supervisor_df=pd.read_sql(querys['Supervisor'],conectando)

vendedor_df=pd.read_sql(querys['Vendedor'],conectando)

In [5]:
temp_df=falta_df.merge(vendedor_df,on='ID Vendedor',how='inner')

temp_df=temp_df.merge(supervisor_df,on='ID Equipe',how='inner')

temp_df.columns

Index(['Data de Falta', 'ID Vendedor', 'ID Cliente', 'Nome Fantasia', 'Pedido',
       'SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',
       'Total do Pedido', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Equipe', 'Supervisor', 'E-mail Sup', 'Gerente',
       'E-mail Gerg', 'Diferente'],
      dtype='object')

In [6]:
temp_df[['Data de Falta', 'ID Vendedor', 'ID Cliente', 'Nome Fantasia', 'Pedido','SKU', 'Produto', 'Unid. VDA', 'Qtde. VDA', 'Valor Unitário',      'Total do Pedido', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail','Categoria', 'Equipe', 'Supervisor']]

,Data de Falta,ID Vendedor,ID Cliente,Nome Fantasia,Pedido,SKU,Produto,Unid. VDA,Qtde. VDA,Valor Unitário,Total do Pedido,Vendedor,Nome Resumido,ID Equipe,E-mail,Categoria,Equipe,Supervisor
0,2022-05-18,MAYRAQS,25555,EMPORIO SAO DIOGO,399495,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,UN,10.00,4.73,47.30,MAYRA QUEIROS DOS SANTOS,MAYRA SANTOS,0009,MAYRA.SANTOS@DEMARCHISAOPAULO.COM.BR,CLT,EQUIPE 9 (ATIVO),ROGERIO FELIPIM
1,2022-05-18,RENATAAP,12320,MERCADO SAO PAULO 20,399333,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,12.00,15.91,190.92,RENATA APARECIDA NOCHIERI COSTA,RENATA COSTA,0009,RENATA@DEMARCHISAOPAULO.COM.BR,CLT,EQUIPE 9 (ATIVO),ROGERIO FELIPIM
2,2022-05-18,VALDO,16611,MONTE ALTO,399569,1002,"CONGELADO MIX DE LEGUMES (1,2KG )",UN,5.00,17.95,82.55,LUIZ CARLOS KAZLAUSKAS VALDO,LUIZ VALDO,0001,LCKVALDO@IG.COM.BR,PJ,EQUIPE 1 (REP),ROGERIO FELIPIM
3,2022-05-18,ELENICEV,7362,COMERCIAL S ETELVINA,399800,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,10.00,15.91,159.10,ELENICE VIRGINIA DAL POGGETTO FLEMING,ELENICE FLEMING,0001,VIRGINIAFLEMING@UOL.COM.BR,PJ,EQUIPE 1 (REP),ROGERIO FELIPIM
4,2022-05-18,CELSOCAR,20669,BONSABOR,399883,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,3.00,15.91,47.73,CELSO CARDOSO AMARO,CELSO AMARO,0001,CELSO.C.AMARO@GMAIL.COM,PJ,EQUIPE 1 (REP),ROGERIO FELIPIM
5,2022-05-18,PEDROCAR,16412,MANTEQUILLA,399315,912,CONGELADO FUNDO DE ALCACHOFRA 400G BND,UN,1.00,20.48,20.48,PEDRO CARBONE FILHO,PEDRO FILHO,0002,PEDROCARBONE1@GMAIL.COM,PJ,EQUIPE 2 (REP),ULISSES BACCI
6,2022-05-18,JOSEFARI,25916,CARNES & CIA,398501,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,20.00,15.91,318.20,JOSE FARIA,JOSE FARIA,0002,JOSELUIZFARIA2011@HOTMAIL.COM,PJ,EQUIPE 2 (REP),ULISSES BACCI
7,2022-05-18,ROBSONAN,25814,CHAMA SUPERMERCADOS,399744,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,2.05,11.99,24.58,ROBSON DOMINGUES DE ANDRADE,ROBSON ANDRADE,0004,ROBSON_DOMINGUES1@HOTMAI.COM,CLT,EQUIPE 4 (VAR),DOUGLAS TEDESCHI
8,2022-05-18,REGIANE,19428,SELMA DELAZARI EIREL,399777,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,CX,21.00,4.73,99.33,REGIANE CARDOSO,REGIANE CARDOSO,0003,REGIANE.DESTROS@GMAIL.COM,CLT,EQUIPE 3 (CLT),ULISSES BACCI
9,2022-05-18,REGIANE,19428,SELMA DELAZARI EIREL,399778,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,CX,21.00,4.73,99.33,REGIANE CARDOSO,REGIANE CARDOSO,0003,REGIANE.DESTROS@GMAIL.COM,CLT,EQUIPE 3 (CLT),ULISSES BACCI


In [7]:
pedidos=len(temp_df['Pedido'].unique().tolist())

total=temp_df['Total do Pedido'].sum()

mix=len(temp_df['SKU'].unique().tolist())

In [8]:
temp_df

,Data de Falta,ID Vendedor,ID Cliente,Nome Fantasia,Pedido,SKU,Produto,Unid. VDA,Qtde. VDA,Valor Unitário,...,Nome Resumido,ID Equipe,E-mail,Categoria,Equipe,Supervisor,E-mail Sup,Gerente,E-mail Gerg,Diferente
0,2022-05-18,MAYRAQS,25555,EMPORIO SAO DIOGO,399495,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,UN,10.00,4.73,...,MAYRA SANTOS,0009,MAYRA.SANTOS@DEMARCHISAOPAULO.COM.BR,CLT,EQUIPE 9 (ATIVO),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,N
1,2022-05-18,RENATAAP,12320,MERCADO SAO PAULO 20,399333,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,12.00,15.91,...,RENATA COSTA,0009,RENATA@DEMARCHISAOPAULO.COM.BR,CLT,EQUIPE 9 (ATIVO),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,N
2,2022-05-18,VALDO,16611,MONTE ALTO,399569,1002,"CONGELADO MIX DE LEGUMES (1,2KG )",UN,5.00,17.95,...,LUIZ VALDO,0001,LCKVALDO@IG.COM.BR,PJ,EQUIPE 1 (REP),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,N
3,2022-05-18,ELENICEV,7362,COMERCIAL S ETELVINA,399800,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,10.00,15.91,...,ELENICE FLEMING,0001,VIRGINIAFLEMING@UOL.COM.BR,PJ,EQUIPE 1 (REP),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,N
4,2022-05-18,CELSOCAR,20669,BONSABOR,399883,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,3.00,15.91,...,CELSO AMARO,0001,CELSO.C.AMARO@GMAIL.COM,PJ,EQUIPE 1 (REP),ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,N
5,2022-05-18,PEDROCAR,16412,MANTEQUILLA,399315,912,CONGELADO FUNDO DE ALCACHOFRA 400G BND,UN,1.00,20.48,...,PEDRO FILHO,0002,PEDROCARBONE1@GMAIL.COM,PJ,EQUIPE 2 (REP),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,S
6,2022-05-18,JOSEFARI,25916,CARNES & CIA,398501,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,20.00,15.91,...,JOSE FARIA,0002,JOSELUIZFARIA2011@HOTMAIL.COM,PJ,EQUIPE 2 (REP),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,S
7,2022-05-18,ROBSONAN,25814,CHAMA SUPERMERCADOS,399744,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",UN,2.05,11.99,...,ROBSON ANDRADE,0004,ROBSON_DOMINGUES1@HOTMAI.COM,CLT,EQUIPE 4 (VAR),DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,S
8,2022-05-18,REGIANE,19428,SELMA DELAZARI EIREL,399777,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,CX,21.00,4.73,...,REGIANE CARDOSO,0003,REGIANE.DESTROS@GMAIL.COM,CLT,EQUIPE 3 (CLT),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,S
9,2022-05-18,REGIANE,19428,SELMA DELAZARI EIREL,399778,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,CX,21.00,4.73,...,REGIANE CARDOSO,0003,REGIANE.DESTROS@GMAIL.COM,CLT,EQUIPE 3 (CLT),ULISSES BACCI,ULISSES@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,S


In [9]:
temp_df[['SKU','Produto','Total do Pedido']].groupby(['SKU','Produto']).sum().reset_index()

,SKU,Produto,Total do Pedido
0,507,"CONGELADO JARDINEIRA S/ MILHO ( 1,2KG )",740.53
1,912,CONGELADO FUNDO DE ALCACHOFRA 400G BND,20.48
2,1002,"CONGELADO MIX DE LEGUMES (1,2KG )",82.55
3,9491,PÃO HB PN10 TRAD C/GERGELIM ARYZTA FP (PCT 4UN...,245.96
